# Importing packages

In [1]:
# from huggingface_hub import notebook_login

# notebook_login()

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import transformers
import pinecone
import langchain

from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationChain
from langchain.schema import BaseOutputParser
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from tqdm.auto import tqdm



from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, StoppingCriteria, StoppingCriteriaList
from peft import LoraConfig, get_peft_model
from datasets import load_dataset, Dataset


import time

In [2]:
device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'

In [3]:
device

'cuda:0'

In [ ]:
# model_name = 'microsoft/phi-2'
# model_name = 'bofenghuang/vigogne-2-7b-instruct'

model_name = 'mistralai/Mistral-7B-v0.1'

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_name,
                                             trust_remote_code=True,
                                             quantization_config=bnb_config,
                                             device_map='auto')

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Fine-tuning

### Freezing the original weights

In [ ]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float16)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x):
    return super().forward(x).to(torch.float16)

model.lm_head = CastOutputToFloat(model.lm_head)

### Setting up the LoRa adapters

In [ ]:
def print_trainable_parameters(model):

  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()

  print(f"Trainable params: {trainable_params} || all params: {all_param} || trainable%: {100*trainable_params/all_param}")

In [ ]:
lora_config = LoraConfig(r=4,    #attention heads
                         lora_alpha=32,
                         lora_dropout=0.01,
                         bias='none',
                         task_type='CAUSAL_LM')

peft_model = get_peft_model(model, lora_config)
print_trainable_parameters(peft_model)

Trainable params: 2097152 || all params: 3502510080 || trainable%: 0.059875687781032735


### Importing data

In [ ]:
qna_df = pd.read_csv('faq_qna_certideal.csv')
qna_df.reset_index(inplace=True)
dataset = Dataset.from_pandas(qna_df)

tokenized_data = dataset.map(lambda samples: tokenizer(samples['Answer']), batched=True)
tokenized_df = tokenized_data.to_pandas()

In [ ]:
qna_df.isnull().sum()

In [ ]:
to_be_dropped = []
for i in tokenized_df.index:
  if len(tokenized_df.loc[i, 'input_ids']) > 512:
    to_be_dropped.append(i)

train_dataset = Dataset.from_pandas(tokenized_df.drop(index=to_be_dropped).reset_index(drop=True))

In [ ]:
# train_args = transformers.TrainingArguments(per_device_train_batch_size=4,
#                                             gradient_accumulation_steps=4,
#                                             max_steps=200,
#                                             fp16=True,
#                                             logging_steps=1,
#                                             output_dir='outputs')


train_args = transformers.TrainingArguments(learning_rate=2e-5,
                                            gradient_accumulation_steps=4,
                                            weight_decay=0.01,
                                            max_steps=10,
                                            push_to_hub=True,
                                            output_dir='outputs')


data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


trainer = transformers.Trainer(model=peft_model,
                               train_dataset=tokenized_data,
                               args=train_args,
                               data_collator=data_collator)


model.config.use_cache = False
trainer.train()

# Conversation chain

### Text generation task

In [ ]:
# list of stopping criteria
stop_token_ids = [tokenizer.convert_tokens_to_ids(x) for x in [['Human', ':'], ['AI', ':']]]
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

# stopping criteria
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
generate_text = pipeline(model=model,
                         tokenizer=tokenizer,
                         return_full_text=True,
                         task='text-generation',
                         stopping_criteria=stopping_criteria,  # without this model rambles during chat
                         temperature=0.2,
                         max_new_tokens=1024,  # max number of tokens to generate in the output
                         repetition_penalty=1.1  # without this output begins repeating
                         )

# template for an instruction with no input
prompt = PromptTemplate(input_variables=["instruction"], template="{instruction}")


llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(memory_key="history",  # important to align
                                        k=10,
                                        #return_messages=True,
                                        return_only_outputs=True  # for conversation chain
                                        )

In [ ]:
class OutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        """Cleans output text"""
        text = text.strip()
        # remove suffix containing "Human:" or "AI:"
        stopwords = ['\nHuman:', '\nAI:']
        for word in stopwords:
            text = text.removesuffix(word)
        return text.strip()

    @property
    def _type(self) -> str:
        """Return output parser type for serialization"""
        return "output_parser"

In [ ]:
parser = OutputParser()

prompt_template = \
"""
Ce qui suit est une conversation amicale entre un humain et une IA.
Cette IA est un assistant pour les potentiels client de CertiDeal.
CertiDeal vends des smartphones reconditionnés. CertiDeal a aussi une offre avec son partenaire Free Mobile.
L'IA répond à toutes les questions concernant CertiDeal, ses produits, ses services et ses partenariats.
Si l’IA ne connaît pas la réponse à une question, elle dit sincèrement qu’elle ne la sait pas.
Les réponses doivent être précises et relativement courtes.

Current conversation:
{history}
Humain: {input}
IA:"""

prompt = PromptTemplate(input_variables=["history", "input"],
                        template=prompt_template,
                        output_parser=parser)

# Chain to have a conversation and load context from memory
chat = ConversationChain(llm=llm,
                         memory=memory,
                         verbose=True,
                         prompt=prompt)

In [ ]:
qna_df = pd.read_csv('/content/drive/Shared drives/Shared Drive/llm/faq_qna_certideal.csv')
dataset = Dataset.from_pandas(qna_df)

### Initialize Vectore Store

In [ ]:
import time

#new code?
from pinecone import Pinecone, ServerlessSpec, PodSpec

pc = Pinecone(api_key='b7ca6249-f785-427c-a93d-0bcac1912cad')

# index = pc.Index(host='<host>')
# get API key from app.pinecone.io and environment from console
# pinecone.init(api_key='b7ca6249-f785-427c-a93d-0bcac1912cad',
#               environment='gcp-starter')

index_name = 'certideal-bloom-rag'

if index_name not in pc.list_indexes().names():
    pc.create_index(index_name,
                    dimension=768,
                    metric='cosine',
                    spec=PodSpec('gcp-starter'))

    # wait for index to finish initialization
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
model_name = "intfloat/multilingual-e5-base"
model_name = "Cohere/Cohere-embed-multilingual-v3.0"

model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

In [ ]:
from langchain.vectorstores import Pinecone

data = dataset.to_pandas()
data = data.reset_index()
data.columns = ['id', 'category', 'question', 'answer']


for i, row in tqdm(data.iterrows()):

  ids = [f"{row['id']}"]
  texts = [row['answer']]
  embeds = embedding.embed_documents(texts)
  metadata = [{'text':row['answer'],
               'question':row['question'],
               'topic':row['category']}]
  index.upsert(vectors=zip(ids, embeds, metadata))


vectorstore = Pinecone(index, embedding.embed_query, 'text')


0it [00:00, ?it/s]

In [ ]:
def augmented_prompt(query:str, vectorstore=vectorstore) -> str:
  """
  Performs similarity search on the vectore database and returns
  an augmented prompt for the Language Model
  """

  res = vectorstore.similarity_search(query, k=3)
  source = '\n'.join([x.page_content for x in res])
  aug_prompt = f"""
  En utilisant les contextes ci-dessous, répondez à la requête.
  Contextes:
  {source}
  Requête:
  {query}
  """
  return aug_prompt

def get_chat_response(query:str, chat=chat, rag=True) -> str:
  """
  Returns an answer to a given question or query
  """

  if chat != None:

    if rag:
      prompt = augmented_prompt(query, vectorstore)
    else:
      prompt = query

    res = chat.predict_and_parse(input=prompt)
    return res

  else:
    print('No chat')

In [ ]:
res = get_chat_response("Parle moi de CertiDeal")
print(res)